In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev2\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost

In [2]:
import math
from datetime import datetime,timedelta
import numpy as np
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR 
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import KNeighborsClassifier

In [3]:
log_base = np.e

In [4]:
df_final = pandas.read_csv('phase2_trainingset.csv')

In [5]:
def delete_abnormal(df, tg, dire, time):
    df_temp = df_final_sorted[(df_final_sorted['tollgate_id'] == tg) & (df_final_sorted['direction'] == dire) &\
                (df_final_sorted['time'] == time)]\
                 .reset_index().drop(['index'],axis=1)
    volume_temp = df_temp['volume']
    mean_temp = volume_temp.mean()
    length = len(df_temp)
    for i in xrange(length):
        if np.abs(volume_temp[i] - mean_temp) > 0.8 * mean_temp:
            df_temp = df_temp.drop(i)
    return df_temp.reset_index().drop('index',axis=1)

In [6]:
df_final_sorted = df_final.sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
df_final_normal = pandas.DataFrame({})
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 1, 0, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 1, 1, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 2, 0, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 3, 0, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 3, 1, i), ignore_index=True)

In [7]:
df_final_normal['date_temp'] = df_final_normal['time_start'].str.split('-',expand=True)[2]\
                               .str.split(' ',expand=True)[0].apply(int)
df_final_sorted['date_temp'] = df_final_sorted['time_start'].str.split('-',expand=True)[2]\
                               .str.split(' ',expand=True)[0].apply(int)
#df_final_normal = df_final_normal[~((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] < 8))]

# six xgboost

In [8]:
df_remain10m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 28)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 30)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 11)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 19)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation10m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 30)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 11)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 19))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df10_appendm = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 30)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df_validation10am = df_validation10m.append(df10_appendm)

df_validation_final10m = df_validation10m.reset_index().drop(['index'],axis=1)
df_remain_final10m = df_remain10m.reset_index().drop(['index'],axis=1)
df_remain_shuffle10m = df_remain_final10m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle10m['volume_log'] = np.log(df_remain_shuffle10m['volume'])/np.log(log_base)
final_training_input10m = df_remain_shuffle10m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input10m = df_validation_final10m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]

In [9]:
df_remain11m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 10)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 11)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 13)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation11m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 10)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 11)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 13)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
# df11_appendm = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
# df_validation11am = df_validation11m.append(df11_appendm)

df_validation_final11m = df_validation11m.reset_index().drop(['index'],axis=1)
df_remain_final11m = df_remain11m.reset_index().drop(['index'],axis=1)
df_remain_shuffle11m = df_remain_final11m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle11m['volume_log'] = np.log(df_remain_shuffle11m['volume'])/np.log(log_base)
final_training_input11m = df_remain_shuffle11m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input11m = df_validation_final11m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]

In [10]:
df_remain20m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 21)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 6)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 20)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation20m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 21)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 6)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 20))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df20_append = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 12)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df_validation20ma = df_validation20m.append(df20_append)

df_validation_final20m = df_validation20m.reset_index().drop(['index'],axis=1)
df_remain_final20m = df_remain20m.reset_index().drop(['index'],axis=1)
df_remain_shuffle20m = df_remain_final20m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle20m['volume_log'] = np.log(df_remain_shuffle20m['volume'])/np.log(log_base)
final_training_input20m = df_remain_shuffle20m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input20m = df_validation_final20m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input20m

In [11]:
df_remain30m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 28)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 20)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation30m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 20))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
df30_appendm1 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)\
                             & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
# df30_appendm2 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
df_validation301m = df_validation30m.append(df30_appendm1)
# df_validation302m = df_validation301m.append(df30_appendm2)

df_validation_final30m = df_validation301m.reset_index().drop(['index'],axis=1)
df_remain_final30m = df_remain30m.reset_index().drop(['index'],axis=1)
df_remain_shuffle30m = df_remain_final30m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle30m['volume_log'] = np.log(df_remain_shuffle30m['volume'])/np.log(log_base)
final_training_input30m = df_remain_shuffle30m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input30m = df_validation_final30m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input30m

In [12]:
df_remain31m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 29)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 29)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 21)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation31m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 21))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df31_appendm1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)\
                             & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
# df31_appendm2 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df_validation311m = df_validation31m.append(df31_appendm1)
# df_validation312m = df_validation311m.append(df31_appendm2)

df_validation_final31m = df_validation311m.reset_index().drop(['index'],axis=1)
df_remain_final31m = df_remain31m.reset_index().drop(['index'],axis=1)
df_remain_shuffle31m = df_remain_final31m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle31m['volume_log'] = np.log(df_remain_shuffle31m['volume'])/np.log(log_base)
final_training_input31m = df_remain_shuffle31m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input31m = df_validation_final31m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input31m

In [13]:
input_dimension = 19
dataset = final_training_input10m.values
validation_set = final_validation_input10m.values
X_val_10m = validation_set[:, 0:input_dimension]
Y_val_10m= validation_set[:, input_dimension]
X_10m = dataset[:, 0:input_dimension]
Y_10m = dataset[:, input_dimension]
Y_log_10m = np.log(Y_val_10m)/np.log(log_base)
# Y_log_10m

In [14]:
input_dimension = 19
dataset = final_training_input11m.values
validation_set = final_validation_input11m.values
X_val_11m = validation_set[:, 0:input_dimension]
Y_val_11m = validation_set[:, input_dimension]
X_11m = dataset[:, 0:input_dimension]
Y_11m = dataset[:, input_dimension]
Y_log_11m = np.log(Y_val_11m)/np.log(log_base)

In [15]:
input_dimension = 19
dataset = final_training_input20m.values
validation_set = final_validation_input20m.values
X_val_20m = validation_set[:, 0:input_dimension]
Y_val_20m = validation_set[:, input_dimension]
X_20m = dataset[:, 0:input_dimension]
Y_20m = dataset[:, input_dimension]
Y_log_20m = np.log(Y_val_20m)/np.log(log_base)
#Y_log_20

In [16]:
input_dimension = 19
dataset = final_training_input30m.values
validation_set = final_validation_input30m.values
X_val_30m = validation_set[:, 0:input_dimension]
Y_val_30m = validation_set[:, input_dimension]
X_30m = dataset[:, 0:input_dimension]
Y_30m = dataset[:, input_dimension]
Y_log_30m = np.log(Y_val_30m)/np.log(log_base)

In [17]:
input_dimension = 19
dataset = final_training_input31m.values
validation_set = final_validation_input31m.values
X_val_31m = validation_set[:, 0:input_dimension]
Y_val_31m = validation_set[:, input_dimension]
X_31m = dataset[:, 0:input_dimension]
Y_31m = dataset[:, input_dimension]
Y_log_31m = np.log(Y_val_31m)/np.log(log_base)

In [18]:
df_remain10a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 21)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 28)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 30)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 20)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation10a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 21)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 30)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 20))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df10_appenda = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 13)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df_validation10aa = df_validation10a.append(df10_appenda)

df_validation_final10a = df_validation10a.reset_index().drop(['index'],axis=1)
df_remain_final10a = df_remain10a.reset_index().drop(['index'],axis=1)
df_remain_shuffle10a = df_remain_final10a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle10a['volume_log'] = np.log(df_remain_shuffle10a['volume'])/np.log(log_base)
final_training_input10a = df_remain_shuffle10a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input10a = df_validation_final10a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
#final_validation_input10a

In [19]:
df_remain11a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 21)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 12)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 23)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation11a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 21)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 12)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 23))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
df11_appenda1 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)\
                             & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
df11_appenda2 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)\
                             & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
df_validation11aa1 = df_validation11a.append(df11_appenda1)
df_validation11aa2 = df_validation11aa1.append(df11_appenda2)

df_validation_final11a = df_validation11aa2.reset_index().drop(['index'],axis=1)
df_remain_final11a = df_remain11a.reset_index().drop(['index'],axis=1)
df_remain_shuffle11a = df_remain_final11a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle11a['volume_log'] = np.log(df_remain_shuffle11a['volume'])/np.log(log_base)
final_training_input11a = df_remain_shuffle11a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input11a = df_validation_final11a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
#final_validation_input11a

In [20]:
df_remain20a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 19)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 29)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 4)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 5)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 21)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation20a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 4)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 5)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 21))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df20_appenda1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df20_appenda2 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 10)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df_validation20aa = df_validation20a.append(df20_appenda1)
# df_validation20aaa = df_validation20aa.append(df20_appenda2)

df_validation_final20a = df_validation20a.reset_index().drop(['index'],axis=1)
df_remain_final20a = df_remain20a.reset_index().drop(['index'],axis=1)
df_remain_shuffle20a = df_remain_final20a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle20a['volume_log'] = np.log(df_remain_shuffle20a['volume'])/np.log(log_base)
final_training_input20a = df_remain_shuffle20a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input20a = df_validation_final20a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input20a

In [21]:
df_remain30a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 20)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 14)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 19)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation30a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 20)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 14)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 19))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
# df30_appenda1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
# df_validation30aa = df_validation30a.append(df30_appenda1)

df_validation_final30a = df_validation30a.reset_index().drop(['index'],axis=1)
df_remain_final30a = df_remain30a.reset_index().drop(['index'],axis=1)
df_remain_shuffle30a = df_remain_final30a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle30a['volume_log'] = np.log(df_remain_shuffle30a['volume'])/np.log(log_base)
final_training_input30a = df_remain_shuffle30a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input30a = df_validation_final30a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input30a

In [22]:
df_remain31a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 19)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 19)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 11)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation31a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 11)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df31_appenda1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)\
                             & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
# df31_appenda2 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df_validation31aa = df_validation31a.append(df31_appenda1)
# df_validation31aaa = df_validation31aa.append(df31_appenda2)

df_validation_final31a = df_validation31aa.reset_index().drop(['index'],axis=1)
df_remain_final31a = df_remain31a.reset_index().drop(['index'],axis=1)
df_remain_shuffle31a = df_remain_final31a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle31a['volume_log'] = np.log(df_remain_shuffle31a['volume'])/np.log(log_base)
final_training_input31a = df_remain_shuffle31a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input31a = df_validation_final31a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input31a

In [23]:
input_dimension = 19
dataset = final_training_input10a.values
validation_set = final_validation_input10a.values
X_val_10a = validation_set[:, 0:input_dimension]
Y_val_10a = validation_set[:, input_dimension]
X_10a = dataset[:, 0:input_dimension]
Y_10a = dataset[:, input_dimension]
Y_log_10a = np.log(Y_val_10a)/np.log(log_base)
# Y_log_10a

In [24]:
input_dimension = 19
dataset = final_training_input11a.values
validation_set = final_validation_input11a.values
X_val_11a = validation_set[:, 0:input_dimension]
Y_val_11a = validation_set[:, input_dimension]
X_11a = dataset[:, 0:input_dimension]
Y_11a = dataset[:, input_dimension]
Y_log_11a = np.log(Y_val_11a)/np.log(log_base)
# Y_log_11a

In [25]:
input_dimension = 19
dataset = final_training_input20a.values
validation_set = final_validation_input20a.values
X_val_20a = validation_set[:, 0:input_dimension]
Y_val_20a = validation_set[:, input_dimension]
X_20a = dataset[:, 0:input_dimension]
Y_20a = dataset[:, input_dimension]
Y_log_20a = np.log(Y_val_20a)/np.log(log_base)
# Y_log_20a

In [26]:
input_dimension = 19
dataset = final_training_input30a.values
validation_set = final_validation_input30a.values
X_val_30a = validation_set[:, 0:input_dimension]
Y_val_30a = validation_set[:, input_dimension]
X_30a = dataset[:, 0:input_dimension]
Y_30a = dataset[:, input_dimension]
Y_log_30a = np.log(Y_val_30a)/np.log(log_base)
# Y_log_30a

In [27]:
input_dimension = 19
dataset = final_training_input31a.values
validation_set = final_validation_input31a.values
X_val_31a = validation_set[:, 0:input_dimension]
Y_val_31a = validation_set[:, input_dimension]
X_31a = dataset[:, 0:input_dimension]
Y_31a = dataset[:, input_dimension]
Y_log_31a = np.log(Y_val_31a)/np.log(log_base)
# Y_log_31a

In [28]:
def mape(y,d):
    c=d.get_label()
    rec=np.sum(np.abs(np.power(log_base,y)-np.power(log_base,c))/np.power(log_base,c))/len(c)
    #rec=np.sum(np.abs(y-c)*1.0/c)/len(c)
    return "mape",rec

In [29]:
xlf_10a = xgboost.XGBRegressor(max_depth=12, 
                        learning_rate=0.005, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.9, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [30]:
xlf_10a.fit(X_10a,Y_10a, eval_metric=mape,verbose = False, eval_set = [(X_val_10a, Y_log_10a)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.005, max_delta_step=0, max_depth=12,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.9)

In [31]:
xlf_11a = xgboost.XGBRegressor(max_depth=10, 
                        learning_rate=0.06, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [32]:
xlf_11a.fit(X_11a,Y_11a, eval_metric=mape,verbose = False, eval_set = [(X_val_11a, Y_log_11a)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.06, max_delta_step=0, max_depth=10,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [33]:
xlf_20a = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.025, 
                        n_estimators=2000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [34]:
xlf_20a.fit(X_20a,Y_20a, eval_metric=mape,verbose = False, eval_set = [(X_val_20a, Y_log_20a)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.025, max_delta_step=0, max_depth=8,
       min_child_weight=8, missing=None, n_estimators=2000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [35]:
xlf_30a = xgboost.XGBRegressor(max_depth=10, 
                        learning_rate=0.01, 
                        n_estimators=2000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [36]:
xlf_30a.fit(X_30a,Y_30a, eval_metric=mape,verbose = False, eval_set = [(X_val_30a, Y_log_30a)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=10,
       min_child_weight=8, missing=None, n_estimators=2000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [37]:
xlf_31a = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.05, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [38]:
xlf_31a.fit(X_31a,Y_31a, eval_metric=mape,verbose = False, eval_set = [(X_val_31a, Y_log_31a)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=8,
       min_child_weight=8, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [39]:
xlf_10m = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.01, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.9, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [40]:
xlf_10m.fit(X_10m,Y_10m, eval_metric=mape,verbose = False, eval_set = [(X_val_10m, Y_log_10m)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=8,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.9)

In [41]:
xlf_11m = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.01, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [42]:
xlf_11m.fit(X_11m,Y_11m, eval_metric=mape,verbose = False, eval_set = [(X_val_11m, Y_log_11m)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=8,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [43]:
xlf_20m = xgboost.XGBRegressor(max_depth=12, 
                        learning_rate=0.06, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [44]:
xlf_20m.fit(X_20m,Y_20m, eval_metric=mape,verbose = False, eval_set = [(X_val_20m, Y_log_20m)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.06, max_delta_step=0, max_depth=12,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [45]:
xlf_30m = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.008, 
                        n_estimators=2000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [46]:
xlf_30m.fit(X_30m,Y_30m, eval_metric=mape,verbose = False, eval_set = [(X_val_30m, Y_log_30m)],early_stopping_rounds=200)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.008, max_delta_step=0, max_depth=8,
       min_child_weight=5, missing=None, n_estimators=2000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [47]:
xlf_31m = xgboost.XGBRegressor(max_depth=6, 
                        learning_rate=0.01, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [48]:
xlf_31m.fit(X_31m,Y_31m, eval_metric=mape,verbose = False, eval_set = [(X_val_31m, Y_log_31m)],early_stopping_rounds=300)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6,
       min_child_weight=8, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [49]:
predict_validation_10m = np.power(log_base,xlf_10m.predict(X_val_10m,ntree_limit=xlf_10m.best_iteration+1))
predict_validation_11m = np.power(log_base,xlf_11m.predict(X_val_11m,ntree_limit=xlf_11m.best_iteration+1))
predict_validation_20m = np.power(log_base,xlf_20m.predict(X_val_20m,ntree_limit=xlf_20m.best_iteration+1))
predict_validation_30m = np.power(log_base,xlf_30m.predict(X_val_30m,ntree_limit=xlf_30m.best_iteration+1))
predict_validation_31m = np.power(log_base,xlf_31m.predict(X_val_31m,ntree_limit=xlf_31m.best_iteration+1))

predict_validation_10a = np.power(log_base,xlf_10a.predict(X_val_10a,ntree_limit=xlf_10a.best_iteration+1))
predict_validation_11a = np.power(log_base,xlf_11a.predict(X_val_11a,ntree_limit=xlf_11a.best_iteration+1))
predict_validation_20a = np.power(log_base,xlf_20a.predict(X_val_20a,ntree_limit=xlf_20a.best_iteration+1))
predict_validation_30a = np.power(log_base,xlf_30a.predict(X_val_30a,ntree_limit=xlf_30a.best_iteration+1))
predict_validation_31a = np.power(log_base,xlf_31a.predict(X_val_31a,ntree_limit=xlf_31a.best_iteration+1))
predict_validation = np.array(list(predict_validation_10m)+list(predict_validation_11m)+list(predict_validation_20m)\
                     +list(predict_validation_30m)+list(predict_validation_31m)+ list(predict_validation_10a)\
                     +list(predict_validation_11a)+list(predict_validation_20a)\
                     +list(predict_validation_30a)+list(predict_validation_31a))
Y_val = np.array(list(Y_val_10m)+list(Y_val_11m)+list(Y_val_20m)+list(Y_val_30m)+list(Y_val_31m)\
                +list(Y_val_10a)+list(Y_val_11a)+list(Y_val_20a)+list(Y_val_30a)+list(Y_val_31a))
#predict_validation = xlf.predict(X_val, ntree_limit=model.best_iteration+1)
print mean_squared_error(Y_val, predict_validation)  
MAPE = (np.abs(Y_val - predict_validation)*1.0/Y_val).mean()
print MAPE

309.824451485
0.150398833954


# prediction

In [50]:
df_weather_test = pandas.read_csv("./dataSet/weather (table 7)_2.csv")

In [51]:
def extract_test_tg_dir(df_current, tg_id, direction):
    df_tg_dir = df_current[(df_current['tollgate_id']==tg_id) & (df_current['direction']==direction)].\
                reset_index().drop(['index'],axis=1)
    return df_tg_dir

In [52]:
df_test_last2h = pandas.read_csv('test2_20min_avg_volume_etc_cartype.csv')
df_test_time_temp = df_test_last2h['time_window'].str.split('[',expand=True)[1].str.split(',',expand=True)[0]
df_test_last2h['time_start'] = df_test_time_temp
df_test2h_1_0 = extract_test_tg_dir(df_test_last2h, 1, 0)
df_test2h_1_1 = extract_test_tg_dir(df_test_last2h, 1, 1)
df_test2h_2_0 = extract_test_tg_dir(df_test_last2h, 2, 0)
df_test2h_3_0 = extract_test_tg_dir(df_test_last2h, 3, 0)
df_test2h_3_1 = extract_test_tg_dir(df_test_last2h, 3, 1)
# df_test2h_1_0

In [53]:
def add_last2h_test_dimension(df_tg_dir, key):
    lv_6_8 = [0]*6
    lv_15_17 = [0]*6
    lv_6_8[0] = []
    lv_6_8[1] = []
    lv_6_8[2] = []
    lv_6_8[3] = []
    lv_6_8[4] = []
    lv_6_8[5] = []
    lv_15_17[0] = []
    lv_15_17[1] = []
    lv_15_17[2] = []
    lv_15_17[3] = []
    lv_15_17[4] = []
    lv_15_17[5] = []
    for i in xrange(len(df_tg_dir)):
        if int(pandas.to_datetime(df_tg_dir['time_start'][i]).hour) < 12:
            lv_6_8[i%6].append(df_tg_dir[key][i])
        else:
            lv_15_17[i%6].append(df_tg_dir[key][i])
    return lv_6_8,lv_15_17

In [54]:
model_insert = {}
type_insert = {}

lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'volume')
lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'volume')
lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'volume')
lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'volume')
lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'volume')
lm = [0]*6
la = [0]*6
l_insert = [0]*6
for i in xrange(6):
    lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
for i in xrange(6):
    la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
for i in xrange(6):
    l_insert[i] = lm[i] + la[i]
    
for j in xrange(8):
    lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'model'+'%d'%j)
    lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'model'+'%d'%j)
    lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'model'+'%d'%j)
    lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'model'+'%d'%j)
    lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'model'+'%d'%j)
    lm = [0]*6
    la = [0]*6
    model_insert[j] = [0]*6
    for i in xrange(6):
        lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
    for i in xrange(6):
        la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
    for i in xrange(6):
        model_insert[j][i] = lm[i] + la[i]
    
for j in xrange(2):
    lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'type'+'%d'%j)
    lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'type'+'%d'%j)
    lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'type'+'%d'%j)
    lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'type'+'%d'%j)
    lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'type'+'%d'%j)
    lm = [0]*6
    la = [0]*6
    type_insert[j] = [0]*6
    for i in xrange(6):
        lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
    for i in xrange(6):
        la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
    for i in xrange(6):
        type_insert[j][i] = lm[i] + la[i]

lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'has_etc1')
lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'has_etc1')
lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'has_etc1')
lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'has_etc1')
lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'has_etc1')
lm = [0]*6
la = [0]*6
has_etc1_insert = [0]*6
for i in xrange(6):
    lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
for i in xrange(6):
    la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
for i in xrange(6):
    has_etc1_insert[i] = lm[i] + la[i]

In [55]:
df_test = pandas.read_csv('./dataSet/phase2_submission_sample_volume.csv')
df_test_temp = df_test['time_window'].str.split('[',expand=True)[1].str.split(',',expand=True)[0]
df_test['time_start'] = df_test_temp
df_test['volume'+'lv0'] = l_insert[0]
df_test['volume'+'lv1'] = l_insert[1]
df_test['volume'+'lv2'] = l_insert[2]
df_test['volume'+'lv3'] = l_insert[3]
df_test['volume'+'lv4'] = l_insert[4]
df_test['volume'+'lv5'] = l_insert[5]
for i in xrange(8):
    df_test['model'+('%d'%i)+'lv0'] = model_insert[i][0]
    df_test['model'+('%d'%i)+'lv1'] = model_insert[i][1]
    df_test['model'+('%d'%i)+'lv2'] = model_insert[i][2]
    df_test['model'+('%d'%i)+'lv3'] = model_insert[i][3]
    df_test['model'+('%d'%i)+'lv4'] = model_insert[i][4]
    df_test['model'+('%d'%i)+'lv5'] = model_insert[i][5]
for i in xrange(2):
    df_test['type'+('%d'%i)+'lv0'] = type_insert[i][0]
    df_test['type'+('%d'%i)+'lv1'] = type_insert[i][1]
    df_test['type'+('%d'%i)+'lv2'] = type_insert[i][2]
    df_test['type'+('%d'%i)+'lv3'] = type_insert[i][3]
    df_test['type'+('%d'%i)+'lv4'] = type_insert[i][4]
    df_test['type'+('%d'%i)+'lv5'] = type_insert[i][5]
df_test['has_etc1'+'lv0'] = has_etc1_insert[0]
df_test['has_etc1'+'lv1'] = has_etc1_insert[1]
df_test['has_etc1'+'lv2'] = has_etc1_insert[2]
df_test['has_etc1'+'lv3'] = has_etc1_insert[3]
df_test['has_etc1'+'lv4'] = has_etc1_insert[4]
df_test['has_etc1'+'lv5'] = has_etc1_insert[5]
df_test_new = df_test.drop(['time_window'],axis=1)
# df_test_new

In [56]:
def holiday_judge_out(df_current, df_timewindow):
    judge_list=[]
    for i in xrange(len(df_current)): 
        start_time = datetime.strptime(df_timewindow[i], "%Y-%m-%d %H:%M:%S")
        if (start_time.month == 10) and ((start_time.day/4) == 0) and (df_current['direction'][i] == 0):
            judge_list.append(1)
        else:
            judge_list.append(0)
    df_current['holiday_ornot_out'] = judge_list
    return df_current

In [57]:
def holiday_judge_in(df_current, df_timewindow):
    judge_list=[]
    for i in xrange(len(df_current)): 
        start_time = datetime.strptime(df_timewindow[i], "%Y-%m-%d %H:%M:%S")
        if (start_time.month == 10) and ((start_time.day == 4) or (start_time.day == 5) or (start_time.day == 6)\
                                    or (start_time.day == 7)) and (df_current['direction'][i] == 1):
            judge_list.append(1)
        else:
            judge_list.append(0)
    df_current['holiday_ornot_in'] = judge_list
    return df_current

In [58]:
def insert_weather(df_current, df_timewindow, df_weather):
    pre_list = []
    wind_list = []
    tempra_list = []
    humidity_list = []
    a = df_timewindow.str.split(' ',expand=True)[0]
    b = df_timewindow.str.split(' ',expand=True)[1]
    c = b.str.split(':',expand=True)[0]
    for i in xrange(len(df_current)):
        date = a[i]
        hour = c[i]
        temp0 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['precipitation']
        temp1 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['wind_speed']
        temp2 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['temperature']
        temp3 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['rel_humidity']
        '''
        if len(temp0) == 0:
            temp0 = 0
        if len(temp1) == 0:
            temp1 = 0
        if len(temp2) == 0:
            temp2 = 0
        if len(temp3) == 0:
            temp3 = 0
        '''
        if ((float(temp0) > 0) and (float(temp0) < 2.5)):
            temp0 = 1
        elif float(temp0) >= 2.5:
            temp0 = 2
        if (float(temp1) <= 0.4):
            temp1 = 0
        elif ((float(temp1) > 0.4) and (float(temp1) <= 1.5)):
            temp1 = 1
        elif ((float(temp1) > 1.5) and (float(temp1) <= 5)):
            temp1 = 2
        elif (float(temp1) > 5):
            temp1 =3
        pre_list.append(float(temp0)) 
        wind_list.append(float(temp1))
        tempra_list.append(float(temp2))
        humidity_list.append(float(temp3))
    df_current['precipitation'] = pre_list
    df_current['wind_speed'] = wind_list
    df_current['temperature'] = tempra_list
    df_current['humidity'] = humidity_list
    return df_current

In [59]:
def transfer_volume(df_temp2):
    df_transfer = df_temp2.drop(['volume'],axis=1)
    df_transfer['volume'] = df_temp2['volume']
    return df_transfer

In [60]:
def add_time_dimension(df_input, df_timewindow):
    m_list=[]
    weekday_list=[]
    time_list=[]
    hour_list = []
    for i in xrange(len(df_input)):
        start_time = datetime.strptime(df_timewindow[i], "%Y-%m-%d %H:%M:%S")
        m_list.append(start_time.month)
        hour_list.append(start_time.hour)
        if str(start_time.date()) == '2016-10-08':
            weekday_list.append(3)
        elif str(start_time.date()) == '2016-10-09':
            weekday_list.append(4)
        else:
            weekday_list.append(start_time.weekday())
        time_list.append(start_time.hour*3 + (start_time.minute/20))
    df_input['month'] = m_list
    df_input['weekday'] = weekday_list
    df_input['time'] = time_list
    df_input['hour'] = hour_list
    return df_input

In [61]:
df_time_temp_test = df_test_new['time_start']
# df_insert_weather_test = insert_weather(df_test_new, df_time_temp_test, df_weather_test)
df_add_time_test = add_time_dimension(df_test_new, df_time_temp_test)
df_judge_out = holiday_judge_out(df_add_time_test, df_time_temp_test)
df_h_judge_test = holiday_judge_in(df_judge_out, df_time_temp_test)
df_final_test = transfer_volume(df_h_judge_test)
# df_final_test

In [62]:
df_final_test['time_hour'] = df_final_test['time_start'].str.split('-',expand=True)[2]\
                             .str.split(' ',expand=True)[1].str.split(':',expand=True)[0].apply(int)
df_final_test['date'] = df_final_test['time_start'].str.split('-',expand=True)[2]\
                             .str.split(' ',expand=True)[0].apply(int)
df_final_test['minute'] = df_final_test['time_start'].str.split('-',expand=True)[2]\
                             .str.split(' ',expand=True)[1].str.split(':',expand=True)[1].apply(int)

In [63]:
df_final_test['travel_time_B1'] = [0] * 420
df_final_test['travel_time_C1'] = [0] * 420
df_final_test['travel_time_A2'] = [0] * 420
df_final_test['travel_time_A3'] = [0] * 420
df_final_test['travel_time_B3'] = [0] * 420
df_final_test['travel_time_C3'] = [0] * 420

In [64]:
def insert_onehot(df, key):
    df_temp = pandas.get_dummies(df[key]) 
    length = df_temp.shape[1]
    df_temp.columns=[i for i in xrange(length)]
    for i in xrange(length):
        df.insert(0, key+'%d'%i, df_temp[i])
    return df

In [65]:
insert_onehot(df_final_test, 'tollgate_id')
insert_onehot(df_final_test, 'month')
insert_onehot(df_final_test, 'weekday')
insert_onehot(df_final_test, 'time')
# insert_onehot(df_final_test, 'precipitation')
# df_final_test

,time11,time10,time9,time8,time7,time6,time5,time4,time3,time2,...,volume,time_hour,date,minute,travel_time_B1,travel_time_C1,travel_time_A2,travel_time_A3,travel_time_B3,travel_time_C3
0,0,0,0,0,0,0,0,0,0,0,...,57.06,8,25,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,261.65,8,26,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,161.31,8,27,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,266.66,8,28,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,183.69,8,29,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,198.50,8,30,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,189.78,8,31,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,67.83,8,25,20,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,190.16,8,26,20,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,115.11,8,27,20,0,0,0,0,0,0


In [66]:
df_final_test['volumelv0_log'] = np.log(df_final_test['volumelv0'])/np.log(log_base)
df_final_test['volumelv1_log'] = np.log(df_final_test['volumelv1'])/np.log(log_base)
df_final_test['volumelv2_log'] = np.log(df_final_test['volumelv2'])/np.log(log_base)
df_final_test['volumelv3_log'] = np.log(df_final_test['volumelv3'])/np.log(log_base)
df_final_test['volumelv4_log'] = np.log(df_final_test['volumelv4'])/np.log(log_base)
df_final_test['volumelv5_log'] = np.log(df_final_test['volumelv5'])/np.log(log_base)
df_final_test['has_etc1lv0_log'] = np.log(df_final_test['has_etc1lv0']+1)/np.log(log_base)
df_final_test['has_etc1lv1_log'] = np.log(df_final_test['has_etc1lv1']+1)/np.log(log_base)
df_final_test['has_etc1lv2_log'] = np.log(df_final_test['has_etc1lv2']+1)/np.log(log_base)
df_final_test['has_etc1lv3_log'] = np.log(df_final_test['has_etc1lv3']+1)/np.log(log_base)
df_final_test['has_etc1lv4_log'] = np.log(df_final_test['has_etc1lv4']+1)/np.log(log_base)
df_final_test['has_etc1lv5_log'] = np.log(df_final_test['has_etc1lv5']+1)/np.log(log_base)
df_final_test['type0lv0_log'] = np.log(df_final_test['type0lv0']+1)/np.log(log_base)
df_final_test['type0lv1_log'] = np.log(df_final_test['type0lv1']+1)/np.log(log_base)
df_final_test['type0lv2_log'] = np.log(df_final_test['type0lv2']+1)/np.log(log_base)
df_final_test['type0lv3_log'] = np.log(df_final_test['type0lv3']+1)/np.log(log_base)
df_final_test['type0lv4_log'] = np.log(df_final_test['type0lv4']+1)/np.log(log_base)
df_final_test['type0lv5_log'] = np.log(df_final_test['type0lv5']+1)/np.log(log_base)
df_final_test['model1lv0_log'] = np.log(df_final_test['model1lv0']+1)/np.log(log_base)
df_final_test['model1lv1_log'] = np.log(df_final_test['model1lv1']+1)/np.log(log_base)
df_final_test['model1lv2_log'] = np.log(df_final_test['model1lv2']+1)/np.log(log_base)
df_final_test['model1lv3_log'] = np.log(df_final_test['model1lv3']+1)/np.log(log_base)
df_final_test['model1lv4_log'] = np.log(df_final_test['model1lv4']+1)/np.log(log_base)
df_final_test['model1lv5_log'] = np.log(df_final_test['model1lv5']+1)/np.log(log_base)

In [67]:
df_test_input_final = df_final_test.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# df_test_input_final

In [68]:
dataset_test = df_test_input_final.values
X_test = dataset_test[:, 0:input_dimension]
# X_test

In [69]:
volume_predict_out1 = np.power(log_base, xlf_10m.predict(X_test[:42],ntree_limit=xlf_10m.best_iteration+1))

In [70]:
volume_predict_out2 = np.power(log_base, xlf_11m.predict(X_test[42:84],ntree_limit=xlf_11m.best_iteration+1))

In [71]:
volume_predict_out3 = np.power(log_base, xlf_20m.predict(X_test[84:126],ntree_limit=xlf_20m.best_iteration+1))

In [72]:
volume_predict_out4 = np.power(log_base, xlf_30m.predict(X_test[126:168],ntree_limit=xlf_30m.best_iteration+1))

In [73]:
volume_predict_out5 = np.power(log_base, xlf_31m.predict(X_test[168:210],ntree_limit=xlf_31m.best_iteration+1))

In [74]:
volume_predict_out6 = np.power(log_base, xlf_10a.predict(X_test[210:252],ntree_limit=xlf_10a.best_iteration+1))

In [75]:
volume_predict_out7 = np.power(log_base, xlf_11a.predict(X_test[252:294],ntree_limit=xlf_11a.best_iteration+1))

In [76]:
volume_predict_out8 = np.power(log_base, xlf_20a.predict(X_test[294:336],ntree_limit=xlf_20a.best_iteration+1))

In [77]:
volume_predict_out9 = np.power(log_base, xlf_30a.predict(X_test[336:378],ntree_limit=xlf_30a.best_iteration+1))

In [78]:
volume_predict_out10 = np.power(log_base, xlf_31a.predict(X_test[378:420],ntree_limit=xlf_31a.best_iteration+1))

In [79]:
volume_predict_out = np.array(list(volume_predict_out1)+list(volume_predict_out2)+list(volume_predict_out3)+\
                             list(volume_predict_out4)+list(volume_predict_out5)+list(volume_predict_out6)+\
                             list(volume_predict_out7)+list(volume_predict_out8)+list(volume_predict_out9)+\
                             list(volume_predict_out10))
# volume_predict_out

In [80]:
df_submission = pandas.read_csv('./dataSet/phase2_submission_sample_volume.csv')
df_submission_new = df_submission.drop(['volume'],axis=1)
df_submission_new['volume'] = volume_predict_out
df_submission_new.to_csv('phase2_submission_volume_06_01.csv',index=False)